- 所有文件都会先经过ocr识别处理
- ocr后的结果应该是一个大的json文件，该文件应：
    - 嵌套式体现文档结构，表明哪些是标题及其标题等级
    - 文档内容应区分表格内容（有开头结尾标记）和其他
    - 内容跨页需要合并
- 由于ocr接正在开发中，本方案依据理想化的ocr结果编写

In [83]:
from paddlenlp import Taskflow
import pandas as pd
import json
from IPython.display import display
import re

# 个人信用报告

## == 内容样例

![个人信用报告](4_简版方案/个人信用报告.png)

In [146]:
grxybg_data = """
\n报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15\n姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚\n其他证件信息：护照G300234234/军人身份证 M09876893\n
"""
grxybg_data

'\n\n报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15\n姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚\n其他证件信息：护照G300234234/军人身份证 M09876893\n\n'

In [147]:
def grxybg_text_preprocess(data):
    data = data.strip().strip("\n").strip()
    data = data.replace("\n", "  ")
    return data

grxybg_data = grxybg_text_preprocess(data=grxybg_data)
grxybg_data

'报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15  姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚  其他证件信息：护照G300234234/军人身份证 M09876893'

## == 预期提取效果

In [ ]:
'''人工提前确定好所有关键字段'''
grxybg_schema = []
grxybg_schema.append(['报告编号', '报告时间'])
grxybg_schema.append(['姓名', '证件类型', '证件号码', '是否已婚'])
grxybg_schema.append(['姓名', '证件类型', '证件号码', '是否已婚'])
grxybg_schema.append(['其他证件信息'])

# 信贷记录

## 信息概要
待补充

### 资产处置信息
略

### 垫款信息 
略

## 信用卡

### 发生过逾期的贷记卡账户明细如下：

#### == 内容样例

![发生过逾期的贷记卡账户明细](./4_简版方案/发生过逾期的贷记卡账户明细.png)

In [139]:
djkyq_data = """
\n1. 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度\n10,000，已变成呆账，余额500。\n2. 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，\n余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超\n过90天。\n3. 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内\n有7个月处于逾期状态，其中3个月逾期超过90天。\n2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。\n
"""
print(djkyq_data)



1. 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度
10,000，已变成呆账，余额500。
2. 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，
余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超
过90天。
3. 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内
有7个月处于逾期状态，其中3个月逾期超过90天。
2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。




In [136]:
"""预处理分割data"""
def djkyq_split_cases(data):  ## 贷记卡逾期
    data = '\n' + data.strip().strip("\n")

    ## 分段标准: 一个或多个空白 + 一个或多个数字 + . + 空格
    split_pattern = r'\s+\d+\.\ '
    data_splited = re.split(split_pattern, data)

    cases = []
    for case in data_splited:
        if case == '':
            continue
        ## 去头尾空格，去所有\n
        case = case.strip().replace("\n","")
        cases.append(case)
    return cases

cases = djkyq_split_cases(data=djkyq_data)
case_1, case_2, case_3 = cases[0], cases[1], cases[2]
print(f"case_1: {case_1} \n")
print(f"case_2: {case_2} \n")
print(f"case_3: {case_3} \n")

case_1: 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度10,000，已变成呆账，余额500。 

case_2: 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。 

case_3: 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。 



#### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：：
```json
{
    'header-1-1-0':{
        'header-name':'发生过逾期的贷记卡账户明细',
        'content': [ocr返回结果]
        'info_extraction':[case_0_info, case_1_info, case_2_info]
    }
}
```
`case_i_info` 的定义如下：

In [77]:
'''人工定好schema内容'''
djkyq_schema = ['发卡日期', '发卡机构','账户币种','卡片尾号','销户日期']
djkyq_schema.extend(['信用额度截至日期','信用额度','是否变成呆账','呆账余额','当前有无逾期'])
djkyq_schema.extend(['近5年处于逾期状态月数','逾期超过90天月数'])  ## ?? '5年'定死、?? '90天'定死

case_i_info = {k:"xxx" for k in djkyq_schema}
case_i_info

{'发卡日期': 'xxx',
 '发卡机构': 'xxx',
 '账户币种': 'xxx',
 '卡片尾号': 'xxx',
 '销户日期': 'xxx',
 '信用额度截至日期': 'xxx',
 '信用额度': 'xxx',
 '是否变成呆账': 'xxx',
 '呆账余额': 'xxx',
 '当前有无逾期': 'xxx',
 '近5年处于逾期状态月数': 'xxx',
 '逾期超过90天月数': 'xxx'}

#### == 提取方案

1. 手动分割段落
1. 人工定义提取关键字段 `djkyq_schema`
   - 待补充 （请业务人员给出建议）
2. paddleNLP taskflow information extraction 用小模型进行字段信息抽取
3. 后续人工用规则矫正
    - 详见下文 `人工修正函数`
    - !! 待补充（请测试/业务人员给出建议）
    

#### == 案例执行结果

------------------------------ 模型抽取 ----------------------------------

In [49]:
""" 加载信息抽取模型 """
## 默认uie_base
djkyq_ie = Taskflow('information_extraction', schema=djkyq_schema)  # 贷记卡逾期

def extract_info_from_taskflow(task, case, schema):
    '''
    task: taskflow
    case: string, info for analysing
    schema: from taskflow
    '''
    info = task(case)  ## 模型抽取信息
    result = {}  ## 存放抽取结果
    for k in schema:
        v = info[0].get(k)
        if v:
            v = v[0].get("text")
        result[k] = v
    return result

[2024-08-23 11:46:44,595] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'C:\Users\0049004320\.paddlenlp\taskflow\information_extraction\uie-base'.


In [110]:
'''
无法识别`是否变成呆账`
'''
print(case_1)
result_1 = extract_info_from_taskflow(task=djkyq_ie, case=case_1, schema=djkyq_schema)
result_1

2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度10,000，已变成呆账，余额500。


{'发卡日期': '2004 年 8 月 30 日',
 '发卡机构': '中国建设银行北京分行',
 '账户币种': '人民币',
 '卡片尾号': '0001',
 '销户日期': None,
 '信用额度截至日期': '2010 年 10 月',
 '信用额度': '10,000',
 '是否变成呆账': None,
 '呆账余额': '500',
 '当前有无逾期': None,
 '近5年处于逾期状态月数': None,
 '逾期超过90天月数': None}

In [111]:
print(case_2)
result_2 = extract_info_from_taskflow(task=djkyq_ie, case=case_2, schema=djkyq_schema)
result_2

2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。


{'发卡日期': '2004年8月30日',
 '发卡机构': '中国工商银行北京分行',
 '账户币种': '人民币',
 '卡片尾号': '0002',
 '销户日期': None,
 '信用额度截至日期': '2016年7月',
 '信用额度': '50,000',
 '是否变成呆账': None,
 '呆账余额': '5,000',
 '当前有无逾期': None,
 '近5年处于逾期状态月数': '11个月',
 '逾期超过90天月数': None}

In [112]:
'''
识别错误：改5年为6年，改90天为80天
'''
case_2_1 = "2.1. 2004 年 8 月 30 日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至 2016 年 7 月，信用额度 50,000，余额 5,000（含未出单的大额专项分期余额 4,000），当前有逾期。最近 6 年内有 11 个月处于逾期状态，其中 5 个月逾期超过 80 天。"
print(case_2_1)
result_2_1 = extract_info_from_taskflow(task=djkyq_ie, case=case_2_1, schema=djkyq_schema)
result_2_1

2.1. 2004 年 8 月 30 日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至 2016 年 7 月，信用额度 50,000，余额 5,000（含未出单的大额专项分期余额 4,000），当前有逾期。最近 6 年内有 11 个月处于逾期状态，其中 5 个月逾期超过 80 天。


{'发卡日期': '2004 年 8 月 30 日',
 '发卡机构': '中国工商银行北京分行',
 '账户币种': '人民币',
 '卡片尾号': '0002',
 '销户日期': None,
 '信用额度截至日期': '2016 年 7 月',
 '信用额度': '50,000',
 '是否变成呆账': None,
 '呆账余额': '5,000',
 '当前有无逾期': None,
 '近5年处于逾期状态月数': '11 个月',
 '逾期超过90天月数': '5 个月'}

In [113]:
print(case_3)
result_3 = extract_info_from_taskflow(task=djkyq_ie, case=case_3, schema=djkyq_schema)
result_3

2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。


{'发卡日期': '2010年4月1日',
 '发卡机构': '中国民生银行信用卡中心',
 '账户币种': '人民币',
 '卡片尾号': '0003',
 '销户日期': '2015年12月',
 '信用额度截至日期': None,
 '信用额度': None,
 '是否变成呆账': None,
 '呆账余额': None,
 '当前有无逾期': None,
 '近5年处于逾期状态月数': '7个月',
 '逾期超过90天月数': '3个月'}

--------------------------------- 人工矫正 -------------------------------

In [65]:
""" 定义人工修正函数 """
def result_fixing(case, result):
    '''
    修正逻辑：
        1. 无法识别`'是否变成呆账'`:
            若含有 `呆账`, 则改对应值为 `是`
        2. 逾期状态识别不一定精确识别原文数字改动：
            若不含有`近5年内`, 则改`'近5年处于逾期状态月数'`对应值为`None`
            若不含有`超过90天`, 则改`'逾期超过90天月数'`对应值为`None`
        3. ...
    '''
    case = case.replace(" ", "")  ## 删空格
    
    if '呆账' in case:
        result["是否变成呆账"] = '是'

    if '近5年内' not in case:
        result["近5年处于逾期状态月数"] = None
    if '超过90天' not in case:
        result["逾期超过90天月数"] = None
        
    return result

In [66]:
result_1 = result_fixing(case=case_1, result=result_1)
result_2 = result_fixing(case=case_2, result=result_2)
result_2_1 = result_fixing(case=case_2_1, result=result_2_1)
result_3 = result_fixing(case=case_3, result=result_3)

------------------------------ 最终效果展示 -----------------------------

In [114]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

cases = [case_1, case_2, case_2_1, case_3]
results = [result_1, result_2, result_2_1, result_3]
for i in range(len(cases)):
    print("--------------")
    print(cases[i])
    display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

--------------
2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度10,000，已变成呆账，余额500。


,key,value
0,发卡日期,2004 年 8 月 30 日
1,发卡机构,中国建设银行北京分行
2,账户币种,人民币
3,卡片尾号,0001
4,销户日期,None
5,信用额度截至日期,2010 年 10 月
6,信用额度,"10,000"
7,是否变成呆账,None
8,呆账余额,500
9,当前有无逾期,None


--------------
2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。


,key,value
0,发卡日期,2004年8月30日
1,发卡机构,中国工商银行北京分行
2,账户币种,人民币
3,卡片尾号,0002
4,销户日期,None
5,信用额度截至日期,2016年7月
6,信用额度,"50,000"
7,是否变成呆账,None
8,呆账余额,"5,000"
9,当前有无逾期,None


--------------
2.1. 2004 年 8 月 30 日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至 2016 年 7 月，信用额度 50,000，余额 5,000（含未出单的大额专项分期余额 4,000），当前有逾期。最近 6 年内有 11 个月处于逾期状态，其中 5 个月逾期超过 80 天。


,key,value
0,发卡日期,2004 年 8 月 30 日
1,发卡机构,中国工商银行北京分行
2,账户币种,人民币
3,卡片尾号,0002
4,销户日期,None
5,信用额度截至日期,2016 年 7 月
6,信用额度,"50,000"
7,是否变成呆账,None
8,呆账余额,"5,000"
9,当前有无逾期,None


--------------
2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。


,key,value
0,发卡日期,2010年4月1日
1,发卡机构,中国民生银行信用卡中心
2,账户币种,人民币
3,卡片尾号,0003
4,销户日期,2015年12月
5,信用额度截至日期,None
6,信用额度,None
7,是否变成呆账,None
8,呆账余额,None
9,当前有无逾期,None


#### == 备注

1. 已忽略最后一行 `2014 年 3 月，该机构声明：该客户委托 XX 公司偿还贷款，因开发公司不按时还款导致出现多次逾期。`
   - （处理逻辑是啥，会有别的类似内容吗）
2. 已忽略 `case_2` 中 `（含未出单的大额专项分期余额 4,000）`
   - （处理逻辑是啥，会有别的括号内容吗）
3. !!! `关键字段定义` 和 `手工矫正` 部分需要专业人员进行补充改正
4. 操作细节需根据ocr返回结果调整

### 透支超过 60 天的准贷记卡账户明细如下：
略

### 从未逾期过的贷记卡及透支未超过 60 天的准贷记卡账户明细如下：
略

## 贷款

### 发生过逾期的账户明细如下：
略

## 其他业务

### 发生过逾期的账户明细如下：
略

## 相关还款责任信息
略

# 非信贷交易记录

```json
{
        'header-2':{
        'header-name':'非信贷交易记录',
        'content':'这部分包含您最近 5 年内的非信贷交易记录。金额类数据均以人民币计算，精确到元。'
        'section':{
            'header-2-0':{...},
            'header-2-1':{...},
            ...
        }
    },
}
```

## 后付费记录

### == 内容样例

![后付费记录](4_简版方案/后付费记录.png)

In [121]:
hffjl_data = []
hffjl_data.append(['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月'])
hffjl_data.append(['业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550'])
hffjl_data.append(['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月'])
hffjl_data.append(['业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
hffjl_data.append(['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月'])
hffjl_data.append(['业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
hffjl_data

[['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月'],
 ['业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550'],
 ['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月'],
 ['业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0'],
 ['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月'],
 ['业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0']]

In [124]:
""" 预处理分割data """
def hffjl_split_cases(data):  ## 后付费记录
    cases = []
    n_line_split = 2
    for i in range(len(data)):
        if i % n_line_split == 0:
            cases.append(data[i])
        else:
            cases[-1].extend(data[i])
    return cases

hffjl_cases = hffjl_split_cases(data=hffjl_data)
hffjl_cases

[['机构名称：中国电信北京分公司',
  '业务类型：固定电话后付费',
  '记账年月：2016年7月',
  '业务开通日期：2012年6月28日',
  '当前缴费状态：欠费',
  '当前欠费金额：550',
  '业务开通日期：2012年6月28日',
  '当前缴费状态：欠费',
  '当前欠费金额：550',
  '业务开通日期：2012年6月28日',
  '当前缴费状态：欠费',
  '当前欠费金额：550'],
 ['机构名称：中国移动北京分公司',
  '业务类型：移动电话后付费',
  '记账年月：2016年6月',
  '业务开通日期：2013年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0',
  '业务开通日期：2013年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0',
  '业务开通日期：2013年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0'],
 ['机构名称：北京自来水公司',
  '业务类型：自来水费',
  '记账年月：2016年6月',
  '业务开通日期：2014年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0',
  '业务开通日期：2014年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0',
  '业务开通日期：2014年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0']]

### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-2-0':{
        'header-name':'后付费记录',
        'content':[ocr返回结果],
        'info_extraction':[case_0_info, case_1_info, case_2_info],
    }
}
```
`case_i_info` 提取效果如下：

In [135]:
'''
此处schema应由规则提取得到，
而非人为规定
'''
hffjl_schema = ["机构名称", "业务类型", "记账年月"]
hffjl_schema.extend(["业务开通日期", "当前缴费状态", "当前欠费金额"])

case_i_info = {k:'...' for k in hffjl_schema}
case_i_info

{'机构名称': '...',
 '业务类型': '...',
 '记账年月': '...',
 '业务开通日期': '...',
 '当前缴费状态': '...',
 '当前欠费金额': '...'}

### == 提取方案

- 手动规则提取
    - 先识别出一个一个案例（e.g. 定死两行为一组）
    - 键值对抽取：" "/": "/":"进行分割, 前面为key, 后面为value

### == 案例执行结果

In [133]:
def hffjl_extract_info_from_caselist(cases):
    results = []
    for case in cases:
        result = {}
        for k_v in case:
            if k_v =='':
                continue
            k_v = re.sub(r'[:：]', '|', k_v)  ## 冒号分割
            k_v = re.split(r'\|+', k_v)
            k = re.sub(r'\s+', '', k_v[0])  ## 去除空白
            v = re.sub(r'\s+', '', k_v[1])
            result[k] = v
        results.append(result)
    return results

hffjl_results = hffjl_extract_info_from_caselist(hffjl_cases)

In [134]:
for i in range (len(hffjl_cases)):
    print("--------------")
    print(hffjl_cases[i])
    display(pd.DataFrame(list(hffjl_results[i].items()), columns=['key', 'value']))

--------------
['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月', '业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550', '业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550', '业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550']


,key,value
0,机构名称,中国电信北京分公司
1,业务类型,固定电话后付费
2,记账年月,2016年7月
3,业务开通日期,2012年6月28日
4,当前缴费状态,欠费
5,当前欠费金额,550


--------------
['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月', '业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0', '业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0', '业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0']


,key,value
0,机构名称,中国移动北京分公司
1,业务类型,移动电话后付费
2,记账年月,2016年6月
3,业务开通日期,2013年1月31日
4,当前缴费状态,正常
5,当前欠费金额,0


--------------
['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月', '业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0', '业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0', '业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0']


,key,value
0,机构名称,北京自来水公司
1,业务类型,自来水费
2,记账年月,2016年6月
3,业务开通日期,2014年1月31日
4,当前缴费状态,正常
5,当前欠费金额,0


### == 备注

1. 具体操作需根据ocr返回结果格式调整

# 公共记录
略 -- 'content'

## 欠税记录
略 -- 思路同 `后付费记录`

- 定死两行一组
- 预期识别关键字段名称
    - 主管税务机关
    - 欠税统计日期
    - 欠税总额
    - 纳税人识别号


## 民事判决记录
略 -- 思路同 `后付费记录`

- 定死5行一组
- 预期识别关键字段名称
    -  立案法院
    -  案号
    -  案由
    -  结案方式
    -  立案日期
    -  判决/调解结果
    -  诉讼标的
    -  判决/调解生效日期
    -  诉讼标的金额

## 强制执行记录
略 -- 思路同 `后付费记录`

- 定死6行一组
- 预期识别关键字段名称
    -  执行法院
    -  案号
    -  执行案由
    -  结案方式
    -  立案日期
    -  案件状态
    -  申请执行标的
    -  已执行标的
    -  申请执行标的金额
    -  已执行标的金额
    -  结案日期

## 行政处罚记录
略 -- 思路同 `后付费记录`

- 定死4行一组
- 预期识别关键字段名称
    - 处罚机构
    - 文书编号
    - 处罚内容
    - 行政复议结果
    - 处罚金额
    - 生效日期
    - 截止日期

# 查询记录
略 -- 'content'

## 机构查询记录明细 
略

## 本人查询记录明细
略